In [1]:
import pandas as pd
import numpy as np
from config import CONFIGS
import os
import os
from datetime import datetime

from utils.processing_functions import explode_columnar_df
from utils.weaviate_client import WeaviateClient

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_distances

# visualization packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# turn off jupyter warnings
import warnings
warnings.filterwarnings('ignore')

ENVIRONMENT = os.environ.get("ENVIRONMENT", "dev")
S3_SCRAPER_BUCKET = CONFIGS["s3_scraper_bucket"]
GAME_CONFIGS = CONFIGS["games"]
RATINGS_CONFIGS = CONFIGS["ratings"]
IS_LOCAL = True if os.environ.get("IS_LOCAL", "False").lower() == "true" else False

read_dir = "data/prod/games/game_dfs_clean/"
sims_dir = "data/prod/games/similarity_files/"

weaviate_client = WeaviateClient()
weaviate_client.connect_weaviate_client_docker()


IS_LOCAL: True

Checking for local config file and evaluating for updates from S3.
Loading config from local


/Users/jennywadkins/.local/share/virtualenvs/boardgamegeek-ZH0FNRKg/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:504: UserWarning: <module 'weaviate.client' from '/Users/jennywadkins/.local/share/virtualenvs/boardgamegeek-ZH0FNRKg/lib/python3.12/site-packages/weaviate/client.py'> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


# Content Similarity Models

### File Setup for All Models

In [2]:
games_df = pd.read_pickle(f"{read_dir}games_clean.pkl")
games_df = games_df.sort_values("BayesAvgRating", ascending=False)[
            0 : 1000
        ].reset_index(drop=True)

id_game_map = {x:y for x,y in zip(games_df['BGGId'],games_df['Name'])}
game_id_map = {y:x for x,y in zip(games_df['BGGId'],games_df['Name'])}
bgg_ids = games_df['BGGId'].values
relevant_df = games_df[['BGGId']]

rag_cleaned_sentences_df = pd.read_pickle(f'{sims_dir}top_1000_cleaned_rag.pkl')
rag_cleaned_sentences_df = relevant_df.merge(rag_cleaned_sentences_df, on="BGGId", how="left")
rag_cleaned_sentences_df = rag_cleaned_sentences_df.drop(columns=["Positive_Components","Positive_Sentences","Negative_Components","Negative_Sentences"], axis=1)
rag_cleaned_sentences_df.columns = [x.lower() for x in rag_cleaned_sentences_df.columns]

all_rag_attributes_df = pd.read_pickle(f"{sims_dir}top_1000_cleaned_rag_with_ratings_extrap.pkl")
all_rag_attributes_df = all_rag_attributes_df.drop(columns=["Description","About","Positive_Components","Negative_Components","Positive_Sentences","Negative_Sentences"])
all_rag_attributes_df.columns = all_rag_attributes_df.columns.str.replace(" ","_")
all_rag_attributes_df.columns = [x.lower() for x in all_rag_attributes_df.columns]
all_rag_attributes_df = rag_cleaned_sentences_df.merge(all_rag_attributes_df, on="bggid", how="left")

rag_pros_cons_only_df = all_rag_attributes_df.drop(columns=["about","description"],axis=1)
rag_pros_cons_only_df = rag_pros_cons_only_df.fillna(0).set_index("bggid")
rag_pros_cons_only_df.columns = [x.lower() for x in rag_pros_cons_only_df.columns]

positive_columns = [x for x in rag_pros_cons_only_df.columns if x.startswith('positive_')]
negative_columns = [x for x in rag_pros_cons_only_df.columns if x.startswith('negative_')]
all_columns_to_produce = positive_columns + negative_columns

models_df_storage = {}
arrays_df_storage = {}


In [3]:
test_games = ["Dominion", "Gloomhaven", "Too Many Bones", "Pandemic", "Splendor", "Viticulture Essential Edition", "Great Western Trail", "Everdell", "Chess", "Azul", "Codenames", "Pandemic Legacy: Season 1"]

game_name = "Chess"
game_id = int(games_df[games_df['Name']==game_name]['BGGId'].values[0])
print(game_name, game_id)

Chess 171


In [4]:
def get_closet_picks_weaviate(df, game_id, model, n_picks=10):
    uuid = df.loc[df['bggid']==game_id]['UUID'].values[0]

    similars = weaviate_client.find_near_objects(collection_name=model, uuid=uuid, limit=50)

    picks = {}

    for item in similars:
        picks[str(item.uuid)]=item.metadata.distance
        
    picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

    picks = picks.merge(df, on='UUID', how='inner')[1:]

    return picks[['bggid','distance']].head(n_picks)

In [5]:
def build_results_for_game(game_name):
    game_id = game_id_map[game_name]
    print(f"\nEvaluation: {game_name} {game_id}")

    single_game_results = {}

    for model, model_df in arrays_df_storage.items():
        print(f"Model: {model}")

        mask = model_df[game_id].sort_values(ascending=True) < 1.0
        picks = model_df[game_id].sort_values(ascending=True)[mask]

        # remove the entry that is the same as the game_id, by index. It is not necessarily entry 0
        picks = picks.drop(game_id)

        single_game_results[model] = picks

    for collection_name, model_df in models_df_storage.items():
        print(f"Model: {collection_name}")

        picks = get_closet_picks_weaviate(df=model_df, game_id=game_id, model=collection_name, n_picks=100)

        single_game_results[collection_name] = {x:y for x,y in zip(picks['bggid'], picks['distance'])}

    results_df = pd.DataFrame(single_game_results)

    results_df = results_df.fillna(1).round(3)
    results_df['average_score'] = results_df.mean(axis=1).round(2)

    results_df['name'] = results_df.index.map(id_game_map)

    return results_df.sort_values("average_score", ascending=True).head(20)

In [6]:
def explode_merge_fill(df, reference_df, fill=False, explode=False):
    if explode:
        df = explode_columnar_df(df)
    df = reference_df.merge(df, on='BGGId', how='left').set_index('BGGId')

    if fill==True:
        df = df.astype(float).replace(0,0.01)
        df = df.fillna(0.01)
    else:
        df = df.fillna(0)
    return df

def get_cosine_distances_df(df):
    cosine_sims = cosine_distances(df).round(3)
    return pd.DataFrame(cosine_sims, columns=bgg_ids, index=bgg_ids)

## Array Models - Themes, Categories, Mechanics, Mechanics, Pros-Cons, Game Family

### Weight

In [ ]:
# model="weight"
# weight_df = games_df[['BGGId', 'GameWeight']].set_index("BGGId").round(2)

# # Create the matrix of absolute differences
# matrix = pd.DataFrame(
#     np.abs(weight_df['GameWeight'].values[:, None] - weight_df['GameWeight'].values),
#     index=weight_df.index,
#     columns=weight_df.index
# ).round(4)

# arrays_df_storage[model] = matrix

# {id_game_map[x]:y for x,y in zip(matrix[game_id].sort_values(ascending=True)[:15].index, matrix[game_id].sort_values(ascending=True)[:15])}

### Game Family

In [ ]:
# model="game_family"

# family_df = explode_merge_fill(games_df[['BGGId', 'Family']], relevant_df, explode=True, fill=False)
# sims_by_id = get_cosine_distances_df(family_df)

# arrays_df_storage[model] = sims_by_id

# results = [id_game_map[x] for x in (sims_by_id[game_id].sort_values(ascending=True)[:10].index)]

# {id_game_map[x]:y for x,y in zip(sims_by_id[game_id].sort_values(ascending=True)[:15].index, sims_by_id[game_id].sort_values(ascending=True)[:15])}

### Themes

In [9]:
model = "themes"

themes_df = pd.read_pickle(f"{read_dir}themes_clean.pkl")
themes_df = explode_merge_fill(themes_df, relevant_df, explode=True, fill=False)
sims_by_id = get_cosine_distances_df(themes_df)

arrays_df_storage[model] = sims_by_id

{id_game_map[x]:y for x,y in zip(sims_by_id[game_id].sort_values(ascending=True)[:15].index, sims_by_id[game_id].sort_values(ascending=True)[:15])}


{'Chess': 0.0,
 'Pagan: Fate of Roanoke': 1.0,
 'Draftosaurus': 1.0,
 'Broom Service': 1.0,
 'Carcassonne: Hunters and Gatherers': 1.0,
 'Queendomino': 1.0,
 'Pax Porfiriana': 1.0,
 'Baseball Highlights: 2045': 1.0,
 'Pathfinder Adventure Card Game: Rise of the Runelords – Base Set': 1.0,
 'Nexus Ops': 1.0,
 'Star Trek: Ascendancy': 1.0,
 'Colosseum': 1.0,
 "Warp's Edge": 1.0,
 'Vinhos': 1.0,
 'Brass: Birmingham': 1.0}

### Categories Only

In [10]:
# TO DO: Add this to the game cleaning script

# category_df = df[['BGGId','Cat:Thematic', 'Cat:Strategy', 'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens']]
# subcats_df = pd.read_pickle(f"{read_dir}subcategories_clean.pkl")
# subcats_df = explode_columnar_df(subcats_df).reset_index()
# category_df = category_df.merge(subcats_df, on="BGGId", how="left")
# category_df = category_df.set_index("BGGId")
# category_df.to_pickle(f"{read_dir}categories_clean.pkl")
# category_df.head()

In [11]:
model = "categories"

cat_df = pd.read_pickle(f"{read_dir}categories_clean.pkl")

cat_df = explode_merge_fill(cat_df, relevant_df, fill=True)

sims_by_id = get_cosine_distances_df(cat_df)

arrays_df_storage[model] = sims_by_id

{id_game_map[x]:y for x,y in zip(sims_by_id[game_id].sort_values(ascending=True)[:15].index, sims_by_id[game_id].sort_values(ascending=True)[:15])}

{'Santorini': 0.0,
 'Ingenious': 0.0,
 'TZAAR': 0.0,
 'Go': 0.0,
 'Hive Pocket': 0.0,
 'DVONN': 0.0,
 'Mahjong': 0.0,
 'ZÈRTZ': 0.0,
 'Onitama': 0.0,
 'YINSH': 0.0,
 'The Duke': 0.0,
 'Chess': 0.0,
 'Blokus': 0.0,
 'Hive': 0.0,
 'Reef': 0.286}

### Mechanics Only

In [12]:
model = "mechanics"

mech_df = pd.read_pickle(f"{read_dir}mechanics_clean.pkl")

mech_df = explode_merge_fill(mech_df, relevant_df, explode=True)
sims_by_id = get_cosine_distances_df(mech_df)

arrays_df_storage[model] = sims_by_id

{id_game_map[x]:y for x,y in zip(sims_by_id[game_id].sort_values(ascending=True)[:15].index, sims_by_id[game_id].sort_values(ascending=True)[:15])}

{'Chess': 0.0,
 'Onitama': 0.27,
 'The Duke': 0.388,
 'Scoville': 0.592,
 'Under Falling Skies': 0.613,
 'Specter Ops': 0.613,
 'Quantum': 0.635,
 'Ricochet Robots': 0.635,
 'The Downfall of Pompeii': 0.691,
 'Mexica': 0.691,
 'Mage Wars Arena': 0.691,
 'Hive': 0.691,
 'Santorini': 0.691,
 'Arcadia Quest: Inferno': 0.691,
 'DVONN': 0.711}

### Pros Cons

In [13]:
model = "pros_cons_only"

sims_by_id = get_cosine_distances_df(rag_pros_cons_only_df)

arrays_df_storage[model] = sims_by_id

{id_game_map[x]:y for x,y in zip(sims_by_id[game_id].sort_values(ascending=True)[:15].index, sims_by_id[game_id].sort_values(ascending=True)[:15])}

{'Chess': 0.0,
 'Go': 0.09,
 'Terra Mystica': 0.102,
 'Lords of Vegas': 0.105,
 'Fury of Dracula (Second Edition)': 0.107,
 'Underwater Cities': 0.11,
 'Flamme Rouge': 0.11,
 'Robinson Crusoe: Adventures on the Cursed Island': 0.111,
 'Formula D': 0.111,
 'Downforce': 0.111,
 'Dominion': 0.111,
 'Middara: Unintentional Malum – Act 1': 0.112,
 'Jaipur': 0.112,
 'Shadows over Camelot': 0.113,
 'Terraforming Mars': 0.113}

### Various Numerical Attributes

In [14]:
various_df = games_df[['BGGId', 'Name',
    'BestPlayers',# 'YearPublished',
       'MfgPlaytime',
       'BayesAvgRating',
       'Rank:strategygames', 'Rank:abstracts', 'Rank:partygames',
       'Rank:wargames', 'Rank:thematic', 'Rank:familygames',
       'Rank:childrensgames', 'Rank:cgs', 'GameWeight']].copy()

# Find the mask where 'Rank:strategygames' is NA
mask = various_df['Rank:strategygames'].isna()

# Assign the index positions only to the rows matching the mask
various_df.loc[mask, 'Rank:strategygames'] = various_df.index[mask]

# various_df['GameAgeYears'] = datetime.now().year - various_df['YearPublished']
# various_df = various_df.drop(columns=['Name','YearPublished'], axis=1)

various_df = various_df.drop(columns=['Name'], axis=1)

various_df.columns = [x.lower() for x in various_df.columns]

various_df.head()

,bggid,bestplayers,mfgplaytime,bayesavgrating,rank:strategygames,rank:abstracts,rank:partygames,rank:wargames,rank:thematic,rank:familygames,rank:childrensgames,rank:cgs,gameweight
0,224517,3,120,8.41053,1,28017,28017,28017,28017,28017,28017,28017,3.8692
1,161936,4,60,8.37415,2,28017,28017,28017,1,28017,28017,28017,2.8297
2,174430,3,120,8.34152,4,28017,28017,28017,2,28017,28017,28017,3.9110
3,342942,2,150,8.33850,3,28017,28017,28017,28017,28017,28017,28017,3.7673
4,363622,2,120,8.25461,4,28017,28017,28017,28017,28017,28017,28017,2.8750


In [15]:
# sns.set(font_scale=1)
# games_c = various_df.drop("Name", axis=1).set_index("BGGId").corr()
# # plot a heat map for all correlations in our data set

# # make our figure
# fig, ax = plt.subplots(figsize=(10, 10))

# # we want our heatmap to not show the upper triangle, which is redundant data
# games_c_mask = np.triu(np.ones_like(games_c, dtype=bool))

# # adjust mask and df to hide center diagonal
# games_c_mask = games_c_mask[1:, :-1]
# corr = games_c.iloc[1:, :-1].copy()

# # color map
# cmap = sns.diverging_palette(220, 20, as_cmap=True)

# # plot heatmap
# sns.heatmap(
#     corr,
#     mask=games_c_mask,
#     annot=True,
#     fmt=".2f",
#     cmap=cmap,
#     vmin=-1,
#     vmax=1,
#     cbar_kws={"shrink": 0.8},
#     square=True,
# )

# # yticks rotate
# plt.yticks(rotation=1)

# # title
# title = "CORRELATION MATRIX\nRanking Categories\n"
# plt.title(title, fontsize=14)

# # plt.savefig('images/heatmap.png')

# plt.show()

In [16]:
various_df = various_df.set_index('bggid')

scaler = MinMaxScaler(feature_range=(0,1))

for col in various_df.columns:
    various_df[col] = scaler.fit_transform(various_df[[col]])

various_df = various_df.fillna(0)

various_df.head(2)

,bestplayers,mfgplaytime,bayesavgrating,rank:strategygames,rank:abstracts,rank:partygames,rank:wargames,rank:thematic,rank:familygames,rank:childrensgames,rank:cgs,gameweight
bggid,,,,,,,,,,,,
224517,0.181818,0.10,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.751198
161936,0.272727,0.05,0.979446,0.000036,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.479042


In [17]:
model = "various_attributes"

sims_byid = get_cosine_distances_df(various_df)

arrays_df_storage[model] = sims_byid

{id_game_map[x]:y for x,y in zip(sims_by_id[game_id].sort_values(ascending=True)[:15].index, sims_by_id[game_id].sort_values(ascending=True)[:15])}

{'Chess': 0.0,
 'Go': 0.09,
 'Terra Mystica': 0.102,
 'Lords of Vegas': 0.105,
 'Fury of Dracula (Second Edition)': 0.107,
 'Underwater Cities': 0.11,
 'Flamme Rouge': 0.11,
 'Robinson Crusoe: Adventures on the Cursed Island': 0.111,
 'Formula D': 0.111,
 'Downforce': 0.111,
 'Dominion': 0.111,
 'Middara: Unintentional Malum – Act 1': 0.112,
 'Jaipur': 0.112,
 'Shadows over Camelot': 0.113,
 'Terraforming Mars': 0.113}

## Vector DB Models

### About Only

In [18]:
# collection_name = "about_only"

# about_df = rag_cleaned_sentences_df[['bggid','about']].copy()

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=True, use_about=True)

# about_df = weaviate_client.add_bgg_collection_batch(df=about_df, collection_name=collection_name, use_about=True)

# about_df.to_pickle(f'{sims_dir}about_df.pkl')

In [19]:
collection_name = "about_only"

about_df = pd.read_pickle(f'{sims_dir}about_df.pkl')
models_df_storage[collection_name] = about_df

picks = get_closet_picks_weaviate(about_df, game_id, model=collection_name, n_picks=10)

picks['name'] = [id_game_map[x] for x in picks['bggid']]

picks

,bggid,distance,name
1,2453,0.205593,Blokus
2,2655,0.209620,Hive
3,188,0.221789,Go
4,31999,0.235672,TZAAR
5,244114,0.246360,Yellow & Yangtze
6,294484,0.251611,Unmatched: Cobble & Fog
7,124742,0.253118,Android: Netrunner
8,21763,0.253460,Mr. Jack
9,7854,0.253993,YINSH
10,156566,0.260023,Lords of Xidit


### About and Description

In [20]:
# collection_name = "about_and_desc"

# desc_df = rag_cleaned_sentences_df[['bggid','about', 'description']].copy()

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=True, use_about=True, use_description=True)

# about_df = weaviate_client.add_bgg_collection_batch(df=desc_df, collection_name=collection_name, use_about=True, use_description=True)

# desc_df.to_pickle(f'{sims_dir}desc_df.pkl')

In [21]:
# collection_name = "about_and_desc"

# desc_df = pd.read_pickle(f'{sims_dir}desc_df.pkl')
# models_df_storage[collection_name] = desc_df

# picks = get_closet_picks_weaviate(desc_df, game_id, model=collection_name, n_picks=10)

# picks['name'] = [id_game_map[x] for x in picks['bggid']]

# picks

### About and Keywords

In [22]:
# collection_name = "about_and_pros_cons"

# about_and_pros_cons = all_rag_attributes_df.drop(columns=['description']).copy()

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=True, use_about=True, attributes=all_columns_to_produce)

# about_df = weaviate_client.add_bgg_collection_batch(df=about_and_pros_cons, collection_name=collection_name, use_about=True, attributes=all_columns_to_produce)

# about_and_pros_cons.to_pickle(f'{sims_dir}about_and_pros_cons.pkl')

In [23]:
collection_name = "about_and_pros_cons"

about_and_pros_cons = pd.read_pickle(f'{sims_dir}about_and_pros_cons.pkl')
models_df_storage[collection_name] = about_and_pros_cons

picks = get_closet_picks_weaviate(about_and_pros_cons, game_id, model=collection_name, n_picks=10)

picks['name'] = [id_game_map[x] for x in picks['bggid']]

picks

,bggid,distance,name
1,2453,0.217975,Blokus
2,2655,0.223180,Hive
3,188,0.233348,Go
4,31999,0.246654,TZAAR
5,124742,0.257665,Android: Netrunner
6,294484,0.261491,Unmatched: Cobble & Fog
7,7854,0.261783,YINSH
8,244114,0.263110,Yellow & Yangtze
9,21763,0.265621,Mr. Jack
10,156566,0.268559,Lords of Xidit


### With Keywords, About, and Desc

In [24]:
# collection_name = "all_attributes"

# # Optional scaling of attributes
# # scaler = MinMaxScaler(feature_range=(0,1))
# # for col in all_columns_to_produce:
# #     df[col] = scaler.fit_transform(df[[col]])

# weaviate_client.create_bgg_collection(collection_name=collection_name, reset=True, use_about=True, use_description=True, attributes=all_columns_to_produce)

# all_attributes_df = weaviate_client.add_bgg_collection_batch(df=all_rag_attributes_df, collection_name=collection_name, use_about=True, use_description=True, attributes=all_columns_to_produce)

# all_attributes_df.to_pickle(f'{sims_dir}all_attributes_df.pkl')

In [25]:
# collection_name = "all_attributes"

# all_attributes_df = pd.read_pickle(f'{sims_dir}all_attributes_df.pkl')
# models_df_storage[collection_name] = all_attributes_df

# picks = get_closet_picks_weaviate(all_attributes_df, game_id, model=collection_name, n_picks=10)

# picks['name'] = [id_game_map[x] for x in picks['bggid']]

# picks

### All Comparisons

In [26]:
models_df_storage.keys(), arrays_df_storage.keys()

(dict_keys(['about_only', 'about_and_pros_cons']),
 dict_keys(['themes', 'categories', 'mechanics', 'pros_cons_only', 'various_attributes']))

In [27]:
test_games

['Dominion',
 'Gloomhaven',
 'Too Many Bones',
 'Pandemic',
 'Splendor',
 'Viticulture Essential Edition',
 'Great Western Trail',
 'Everdell',
 'Chess',
 'Azul',
 'Codenames',
 'Pandemic Legacy: Season 1']

In [28]:
build_results_for_game("Too Many Bones")


Evaluation: Too Many Bones 192135
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_attributes
Model: about_only
Model: about_and_pros_cons


,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
235802,0.134,0.000,0.298,0.124,0.005,0.288,0.290,0.16,Too Many Bones: Undertow
174430,0.134,0.179,0.612,0.134,0.005,0.248,0.244,0.22,Gloomhaven
291457,0.134,0.179,0.682,0.140,0.002,0.241,0.244,0.23,Gloomhaven: Jaws of the Lion
295770,0.134,0.179,0.608,0.109,0.002,0.272,0.275,0.23,Frosthaven
169427,0.134,0.490,0.440,0.110,0.087,0.220,0.220,0.24,Middara: Unintentional Malum – Act 1
322708,0.250,0.490,0.298,0.112,0.094,0.257,0.266,0.25,Descent: Legends of the Dark
170771,0.134,0.490,0.510,0.129,0.083,0.239,0.244,0.26,Sword & Sorcery
181530,0.134,0.286,0.600,0.120,0.098,0.294,0.292,0.26,Runebound (Third Edition)
195421,0.134,0.286,0.666,0.122,0.082,0.249,0.254,0.26,Near and Far
181521,0.134,0.490,0.476,0.126,0.107,0.251,0.259,0.26,Warhammer Quest: The Adventure Card Game


In [29]:
build_results_for_game("Gloomhaven")


Evaluation: Gloomhaven 174430
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_attributes
Model: about_only
Model: about_and_pros_cons


,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
291457,0.000,0.000,0.128,0.128,0.001,0.205,0.200,0.09,Gloomhaven: Jaws of the Lion
295770,0.000,0.000,0.199,0.120,0.005,0.266,0.260,0.12,Frosthaven
96848,0.000,0.000,0.380,0.137,0.009,0.270,0.269,0.15,Mage Knight Board Game
169427,0.000,0.179,0.423,0.113,0.106,0.234,0.228,0.18,Middara: Unintentional Malum – Act 1
269385,0.134,0.179,0.518,0.129,0.089,0.208,0.208,0.21,The Lord of the Rings: Journeys in Middle-Earth
17226,0.000,0.179,0.622,0.141,0.113,0.217,0.220,0.21,Descent: Journeys in the Dark
170771,0.000,0.179,0.646,0.126,0.104,0.238,0.233,0.22,Sword & Sorcery
192135,0.134,0.179,0.612,0.134,0.005,0.248,0.244,0.22,Too Many Bones
248562,0.000,0.411,0.501,0.121,0.068,0.201,0.203,0.22,Mage Knight: Ultimate Edition
322708,0.134,0.179,0.557,0.123,0.113,0.262,0.264,0.23,Descent: Legends of the Dark


In [30]:
all_game_results = {}
for game_name in test_games:
    all_game_results[game_name] = build_results_for_game(game_name)


Evaluation: Dominion 36218
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_attributes
Model: about_only
Model: about_and_pros_cons

Evaluation: Gloomhaven 174430
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_attributes
Model: about_only
Model: about_and_pros_cons

Evaluation: Too Many Bones 192135
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_attributes
Model: about_only
Model: about_and_pros_cons

Evaluation: Pandemic 30549
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_attributes
Model: about_only
Model: about_and_pros_cons

Evaluation: Splendor 148228
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_attributes
Model: about_only
Model: about_and_pros_cons

Evaluation: Viticulture Essential Edition 183394
Model: themes
Model: categories
Model: mechanics
Model: pros_cons_only
Model: various_att

In [31]:
all_game_results["Dominion"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
209418,0.000,0.000,0.000,0.139,0.001,0.126,0.129,0.06,Dominion (Second Edition)
40834,0.000,0.000,0.087,0.133,0.000,0.153,0.159,0.08,Dominion: Intrigue
244115,1.000,0.000,0.087,0.126,0.010,0.210,0.212,0.24,Shards of Infinity
147020,1.000,0.000,0.134,0.141,0.001,0.232,0.229,0.25,Star Realms
367041,0.293,0.286,0.691,0.149,0.023,0.242,0.236,0.27,Scholars of the South Tigris
69789,1.000,0.000,0.293,0.118,0.012,0.262,0.258,0.28,Ascension: Deckbuilding Game
108745,1.000,0.000,0.500,0.125,0.004,0.218,0.214,0.29,Seasons
107529,0.000,0.490,0.877,0.113,0.081,0.268,0.269,0.30,Kingdom Builder
68425,1.000,0.000,0.508,0.168,0.013,0.234,0.234,0.31,Eminent Domain
318184,1.000,0.000,0.508,0.144,0.012,0.252,0.249,0.31,Imperium: Classics


In [32]:
all_game_results["Gloomhaven"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
291457,0.000,0.000,0.128,0.128,0.001,0.205,0.200,0.09,Gloomhaven: Jaws of the Lion
295770,0.000,0.000,0.199,0.120,0.005,0.266,0.260,0.12,Frosthaven
96848,0.000,0.000,0.380,0.137,0.009,0.270,0.269,0.15,Mage Knight Board Game
169427,0.000,0.179,0.423,0.113,0.106,0.234,0.228,0.18,Middara: Unintentional Malum – Act 1
269385,0.134,0.179,0.518,0.129,0.089,0.208,0.208,0.21,The Lord of the Rings: Journeys in Middle-Earth
17226,0.000,0.179,0.622,0.141,0.113,0.217,0.220,0.21,Descent: Journeys in the Dark
170771,0.000,0.179,0.646,0.126,0.104,0.238,0.233,0.22,Sword & Sorcery
192135,0.134,0.179,0.612,0.134,0.005,0.248,0.244,0.22,Too Many Bones
248562,0.000,0.411,0.501,0.121,0.068,0.201,0.203,0.22,Mage Knight: Ultimate Edition
322708,0.134,0.179,0.557,0.123,0.113,0.262,0.264,0.23,Descent: Legends of the Dark


In [33]:
all_game_results["Everdell"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
319422,0.000,0.179,0.147,0.126,0.088,0.287,0.299,0.16,Everdell: Collector's Edition
332398,0.000,0.000,0.273,0.163,0.004,0.346,0.356,0.16,Everdell: The Complete Collection
314491,0.711,0.000,0.316,0.140,0.008,0.347,0.341,0.27,Meadow
21882,0.293,0.179,0.730,0.116,0.032,0.352,0.348,0.29,Blue Moon City
241724,0.711,0.179,0.397,0.126,0.102,0.333,0.331,0.31,Villagers
385529,0.646,0.000,0.652,0.126,0.023,0.360,0.356,0.31,The Vale of Eternity
262712,0.646,0.179,0.574,0.130,0.075,0.324,0.319,0.32,Res Arcana
192458,0.750,0.179,0.508,0.126,0.085,0.347,0.345,0.33,51st State: Master Set
385761,0.646,0.179,0.548,0.135,0.091,0.357,0.347,0.33,Faraway
400314,0.423,0.411,0.631,0.151,0.082,0.358,0.360,0.35,Apiary


In [34]:
all_game_results["Viticulture Essential Edition"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
128621,0.000,0.000,0.324,0.099,0.009,0.210,0.217,0.12,Viticulture
42052,0.184,0.000,0.733,0.147,0.027,0.318,0.333,0.25,Vinhos
104006,0.184,0.000,0.622,0.158,0.006,0.395,0.413,0.25,Village
175640,0.500,0.000,0.782,0.124,0.009,0.213,0.220,0.26,Vinhos: Deluxe Edition
39683,0.184,0.000,0.691,0.108,0.015,0.417,0.437,0.26,At the Gates of Loyang
251219,0.293,0.000,0.733,0.099,0.011,0.379,0.396,0.27,Istanbul: Big Box
159675,0.293,0.000,0.691,0.134,0.007,0.384,0.403,0.27,Fields of Arle
196340,0.293,0.000,0.733,0.116,0.004,0.352,0.369,0.27,Yokohama
148949,0.293,0.286,0.622,0.149,0.071,0.347,0.359,0.30,Istanbul
182874,0.500,0.000,0.714,0.140,0.003,0.383,0.396,0.31,Grand Austria Hotel


In [35]:
all_game_results["Pandemic"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
150658,0.500,0.000,0.283,0.119,0.007,0.254,0.251,0.20,Pandemic: The Cure
198928,0.293,0.286,0.244,0.117,0.070,0.252,0.250,0.22,Iberia
161936,0.184,0.490,0.163,0.125,0.167,0.225,0.225,0.23,Pandemic Legacy: Season 1
370913,1.000,0.000,0.198,0.121,0.014,0.261,0.258,0.26,Star Wars: The Clone Wars
248490,1.000,0.000,0.691,0.108,0.008,0.251,0.246,0.33,Atlantis Rising (Second Edition)
234671,1.000,0.286,0.332,0.142,0.086,0.250,0.246,0.33,Pandemic: Rising Tide
314040,1.000,0.490,0.163,0.117,0.157,0.230,0.223,0.34,Pandemic Legacy: Season 0
307002,1.000,0.179,0.662,0.121,0.011,0.239,0.228,0.35,Regicide
136063,1.000,0.286,0.564,0.117,0.078,0.260,0.246,0.36,Forbidden Desert
334986,1.000,0.286,0.537,0.115,0.088,0.268,0.259,0.36,Daybreak


In [36]:
all_game_results["Splendor"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
364073,0.184,0.179,0.106,0.151,0.073,0.210,0.220,0.16,Splendor Duel
293296,0.592,0.000,0.000,0.150,0.004,0.334,0.345,0.20,Splendor: Marvel
232832,0.500,0.179,0.493,0.126,0.071,0.258,0.262,0.27,Century: Golem Edition
241724,1.000,0.000,0.329,0.125,0.008,0.278,0.283,0.29,Villagers
379629,1.000,0.000,0.329,0.117,0.005,0.321,0.322,0.30,Knarr
385761,1.000,0.000,0.329,0.136,0.001,0.336,0.343,0.31,Faraway
274960,1.000,0.000,0.368,0.118,0.005,0.320,0.325,0.31,Point Salad
316377,0.646,0.179,0.493,0.142,0.075,0.341,0.355,0.32,7 Wonders (Second Edition)
367220,1.000,0.000,0.553,0.147,0.002,0.341,0.346,0.34,Sea Salt & Paper
148949,0.293,0.490,0.702,0.107,0.075,0.338,0.342,0.34,Istanbul


In [37]:
all_game_results["Great Western Trail"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
341169,0.000,0.0,0.118,0.110,0.000,0.276,0.281,0.11,Great Western Trail: Second Edition
364011,0.184,0.0,0.065,0.131,0.009,0.299,0.303,0.14,Great Western Trail: Argentina
380607,0.184,0.0,0.143,0.147,0.006,0.300,0.306,0.16,Great Western Trail: New Zealand
196340,0.423,0.0,0.733,0.147,0.007,0.250,0.249,0.26,Yokohama
140620,0.592,0.0,0.564,0.134,0.014,0.309,0.313,0.28,Lewis & Clark: The Expedition
262215,0.423,0.0,0.748,0.141,0.025,0.324,0.329,0.28,Blackout: Hong Kong
39938,0.333,0.0,1.000,0.146,0.030,0.271,0.269,0.29,Carson City
306040,0.742,0.0,0.496,0.134,0.031,0.299,0.300,0.29,Merv: The Heart of the Silk Road
124361,0.667,0.0,0.581,0.144,0.003,0.317,0.323,0.29,Concordia
276025,0.711,0.0,0.555,0.145,0.002,0.307,0.314,0.29,Maracaibo


In [38]:
all_game_results["Chess"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
36235,1.0,0.000,0.388,0.144,0.009,0.280,0.294,0.30,The Duke
188,1.0,0.000,0.711,0.090,0.004,0.222,0.233,0.32,Go
2655,1.0,0.000,0.691,0.138,0.009,0.210,0.223,0.32,Hive
31999,1.0,0.000,0.711,0.132,0.006,0.236,0.247,0.33,TZAAR
2346,1.0,0.000,0.711,0.128,0.005,0.271,0.284,0.34,DVONN
2453,1.0,0.000,0.846,0.119,0.021,0.206,0.218,0.34,Blokus
528,1.0,0.000,0.764,0.134,0.007,0.273,0.281,0.35,ZÈRTZ
7854,1.0,0.000,0.764,0.139,0.006,0.254,0.262,0.35,YINSH
154597,1.0,0.000,0.846,0.133,0.011,0.274,0.289,0.36,Hive Pocket
9674,1.0,0.000,1.000,0.132,0.014,0.273,0.280,0.39,Ingenious


In [39]:
all_game_results["Azul"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
287954,0.293,0.000,0.000,0.128,0.003,0.178,0.178,0.11,Azul: Summer Pavilion
346965,0.000,0.000,0.106,0.173,0.023,0.237,0.235,0.11,Azul: Queen's Garden
256226,0.000,0.000,0.087,0.150,0.082,0.301,0.297,0.13,Azul: Stained Glass of Sintra
363247,1.000,0.000,0.106,0.171,0.023,0.297,0.287,0.27,Azul: Master Chocolatier
283155,1.000,0.000,0.255,0.132,0.004,0.272,0.278,0.28,Calico
163412,1.000,0.000,0.662,0.144,0.001,0.301,0.302,0.34,Patchwork
284435,1.000,0.000,0.742,0.124,0.014,0.321,0.317,0.36,Nova Luna
244228,1.000,0.179,0.553,0.169,0.093,0.284,0.291,0.37,Reef
357563,1.000,0.411,0.324,0.152,0.082,0.316,0.316,0.37,Akropolis
199561,0.293,0.000,0.404,0.121,0.002,1.000,1.000,0.40,Sagrada


In [40]:
all_game_results["Codenames"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
198773,0.000,0.179,0.000,0.144,0.004,0.344,0.351,0.15,Codenames: Pictures
224037,0.000,0.327,0.200,0.147,0.147,0.229,0.240,0.18,Codenames: Duet
225694,0.293,0.179,0.329,0.138,0.002,0.307,0.317,0.22,Decrypto
41114,0.293,0.327,0.576,0.122,0.004,0.453,0.452,0.32,The Resistance
188834,0.423,0.327,0.662,0.121,0.004,0.451,0.461,0.35,Secret Hitler
329839,1.000,0.179,0.684,0.159,0.004,0.290,0.294,0.37,So Clover!
275467,1.000,0.131,0.553,0.121,0.150,0.383,0.388,0.39,Letter Jam
256788,1.000,0.179,0.662,0.122,0.011,0.384,0.385,0.39,Detective Club
36553,1.000,0.411,0.329,0.123,0.007,0.447,0.449,0.40,Time's Up! Title Recall!
156546,1.000,0.327,0.635,0.141,0.004,0.359,0.368,0.40,Monikers


In [41]:
all_game_results["Pandemic Legacy: Season 1"]

,themes,categories,mechanics,pros_cons_only,various_attributes,about_only,about_and_pros_cons,average_score,name
221107,0.423,0.000,0.087,0.128,0.010,0.154,0.148,0.14,Pandemic Legacy: Season 2
314040,1.000,0.000,0.000,0.150,0.009,0.238,0.239,0.23,Pandemic Legacy: Season 0
30549,0.184,0.490,0.163,0.125,0.167,0.225,0.225,0.23,Pandemic
192153,1.000,0.000,0.163,0.156,0.046,0.274,0.274,0.27,Reign of Cthulhu
174430,1.000,0.179,0.621,0.128,0.006,0.274,0.269,0.35,Gloomhaven
156858,1.000,0.286,0.452,0.131,0.114,0.256,0.257,0.36,Black Orchestra
291457,1.000,0.179,0.637,0.137,0.007,0.271,0.269,0.36,Gloomhaven: Jaws of the Lion
295770,1.000,0.179,0.627,0.134,0.017,0.267,0.264,0.36,Frosthaven
257499,1.000,0.286,0.613,0.136,0.102,0.219,0.219,0.37,Arkham Horror (Third Edition)
12493,1.000,0.179,0.763,0.166,0.031,0.231,0.231,0.37,Twilight Imperium: Third Edition
